# Lab03: Web Crawler (Continue) & Information Retrieval.

- MSSV: 19120689
- Họ và tên: Lại Khánh Toàn

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; từ `TODO` cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Sau đó, tạo thư mục có tên `MSSV` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) Chép file notebook, file `t_data.txt` và file `raw_data` của các bạn (nếu file này kích thước lớn các bạn có thể chép link vào `link_data.txt`), nén thư mục `MSSV` này lại và nộp trên moodle.

**Nội dung bài tập**

Cài đặt một web crawler để thu thập dữ liệu từ: https://en.wikipedia.org/wiki/Web_mining.

## Nội dung bài tập

Cài đặt một Web crawler đơn giản bắt đầu từ URL: https://en.wikipedia.org/wiki/Web_mining, tìm liên kết và thu thập dữ liệu trong HTML tại URL này sau đó lặp lại với các URL vừa tìm được.


In [1]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string
import pickle

## 1. Thu thập đường dẫn

- Robot.txt:  https://en.wikipedia.org/robots.txt
- **Bước 1**: Thu thập đường dẫn từ https://en.wikipedia.org/wiki/Web_mining. Lưu trữ vào một danh sách `url_list`. 
- **Bước 2**: Lặp lại bước 1 cho các đường dẫn trong `url_list` (**lưu ý:** kiểm tra các đường dẫn vừa thu được đã nằm trong `url_list` hay không?). Dừng khi đã thu thập được 200 URLs.

In [5]:
def get_urls(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    urls = []
    
    # TODO
    # Lấy các url nằm trong trang web của url này, lưu lại vào biến urls
    for link in soup.find_all('a', href= re.compile('http[s]?://(?:[-\w.])+')):
        urls.append(link.get('href'))
    return urls


def get_urls_recursive(start_url, limit):
    urls = [start_url]
    for url in urls:
        # TODO
        # Lấy các url nằm trong trang web của url này, lưu lại vào biến new_urls
        # Với mỗi url mới trong new_urls:
        #   Nếu nó chưa nằm trong urls thì thêm nó vô  
        # Nếu kích thước của urls vượt quá limit thì dừng và xóa phần dư thừa
        new_urls = get_urls(url)
        for url in new_urls:
            if url not in urls:
                urls.append(url)
        if len(urls) > limit:
            urls = urls[:limit]
            break
    return urls
url_list = get_urls_recursive('https://en.wikipedia.org/wiki/Web_mining', 90)

## 2. Thu thập dữ liệu
Thu thập dữ liệu từ `url_list`. Lưu trữ dữ liệu thu được vào dictionary data với keys là các từ, values gồm 2 phần tử: 
- `url_idx_list` với $idx \in \left[0,200\right) \cap \mathbb{N}$
- `frequency` 
    
Ví dụ: `data['at']=[url_idx_list,frequency]`:
- `url_idx_list`: danh sách các url mà trong dữ liệu của chúng (html document) chứa từ "at". 
- `frequency`: tần suất xuất hiện (số lần xuất hiện) của từ `at` trong dữ liệu của **tất cả đường dẫn thu được**.   

In [3]:
def text_filter(element):
    
    # TODO
    # Cài đặt lại như Lab02
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

def wordList(url):
    
    # TODO
    # Cài đặt lại như Lab02
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    text = soup.findAll(text=True)
    filtered_text = list(filter(text_filter, text)) # list của các chuỗi
    word_list = []

    for strg in filtered_text: 
      new_strg = strg.replace(string.punctuation, " ")
      words = new_strg.split()
      word_list.extend(words)

    return word_list

def read_url(url, url_idx, data):
    # TODO
    # Cài đặt lại như Lab02
    word_list = wordList(url)

    for word in word_list:
      if word not in data:
        data[word] = [[url_idx], 1]
      else:
        if url_idx not in data[word][0]:
          data[word][0].append(url_idx)
        data[word][1]+=1

In [6]:
data = {}
for url_index, url in enumerate(url_list, 1):
    read_url(url, url_index, data)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


## 3. Tiền xử lý
Loại bỏ các item trong data mà key là các stopword.

**Ngữ liệu:** 

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')
print(english_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data]   Unzipping corpora\stopwords.zip.


In [8]:
# TODO
# Loại bỏ các key của biến data mà nằm trong danh sách english_stopwords
for val in list(data):
    if val in english_stopwords:
        del data[val]

## 4. Lưu trữ và biểu diễn dữ liệu
Sử dụng pickle lưu lại data với tên file raw_data.
### 4.1 Cơ sở dữ liệu giao tác:
Thông thường, các cơ sở dữ liệu giao tác được lưu trong flat files (các tập phẳng) thay vì trong một hệ cơ sở dữ liệu. Các item là các số nguyên không âm, mỗi giao tác tương ứng với một dòng các số nguyên phân tách nhau bằng khoảng trắng.
Ví dụ:

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 

32 69 

48 70 71 72 

39 73 74 75 76 77 78 79 

36 38 39 41 48 79 80 81 

82 83 84 

41 85 86 87 88 

39 48 89 90 91 92 93 94 95 96 97 98 99 100 101 

36 38 39 48 89 

39 41 102 103 104 105 106 107 108 

38 39 41 109 110 

39 111 112 113 114 115 116 117 118 

119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 

48 134 135 136 

39 48 137 138 139 140 141 142 143 144 145 146 147 148 149 

39 150 151 152 

38 39 56 153 154 155 

In [ ]:
with open('raw_data', 'wb') as f:
    # TODO
    pickle.dump(data, f)

### 4.2 Xuất dataset
Lưu một cơ sở dữ liệu giao tác (transactional database) vào file t_data.txt: 
- Các item tương ứng với url_idx
- Mỗi transaction tương ứng với một từ.

In [ ]:
with open('t_data.txt', 'w') as f:
    for word, (url_list, freq) in data.items():
        print(*url_list, file=f)

## 5. Truy vấn and, or, not
Ví dụ: 
- Truy vấn `and` câu `web mining`: trả về đường dẫn tới các trang web có cả 2 từ web và từ mining. 
- Truy vấn `or` câu `web mining`: trả về đường dẫn tới các trang web có từ web hoặc từ mining.
- Truy vấn `not` câu `web mining`: trả về đường dẫn tới các trang không có cả từ web và từ mining.

*GỢI Ý: TÁCH CÂU TRUY VẤN THÀNH CÁC TỪ TƯƠNG TỰ PHƯƠNG PHÁP LÀM Ở LAB02.*

In [12]:
ret=[]
def andRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have all terms: urls
    ### if len(ret)==0:  return urls
    ### else update ret with urls: intersection of ret and urls 
    terms = sentence.split(' ')
    
    for term in terms:
        if term not in data:
            return []
    urls = data[terms[0]][0]
    for term in terms[1:]:
        urls = [url for url in urls if url in data[term][0]]
    if len(ret) == 0:
        return urls
    else:
        return [url for url in ret if url in urls]



def orRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have all terms: urls
    ### find urls have at least 1 term: urls
    ### update ret with urls: extend ret with urls
    terms = sentence.split(' ')
    for term in terms:
        if term not in data:
            return []
    urls = data[terms[0]][0]
    for term in terms[1:]:
        urls = [url for url in urls if url in data[term][0]]

    for term in terms:
        if term in data:
            if data[term][0] not in urls:
                urls.extend(data[term][0])
    if len(ret) == 0:
        return urls
    else:
        return ret.extend(urls)


def notRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have at least 1 term: urls
    ### update ret with urls: remove urls from ret 
    terms = sentence.split(' ')
    urls = []
    for term in terms:
        if term in data:
            if data[term][0] not in urls:
                urls.extend(data[term][0])
    if len(ret) == 0:
        return urls
    else:
        return [url for url in ret if url not in urls]

In [13]:
print(andRetrieval([], 'web mining'))
print(orRetrieval([], 'web mining'))
print(notRetrieval(url_list, 'web mining'))

[1, 8, 18, 24, 51, 76, 84]
[1, 8, 18, 24, 51, 76, 84, 1, 2, 8, 15, 18, 24, 51, 76, 84, 1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 18, 20, 22, 23, 24, 25, 26, 27, 33, 34, 35, 36, 37, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 60, 61, 62, 63, 64, 68, 69, 76, 79, 80, 81, 83, 84, 89, 90]
['https://en.wikipedia.org/wiki/Web_mining', 'https://donate.wikimedia.org/wiki/Special:FundraiserRedirector?utm_source=donate&utm_medium=sidebar&utm_campaign=C13_en.wikipedia.org&uselang=en', 'https://ar.wikipedia.org/wiki/%D8%AA%D9%86%D9%82%D9%8A%D8%A8_%D9%81%D9%8A_%D8%A7%D9%84%D8%A8%D9%8A%D8%A7%D9%86%D8%A7%D8%AA', 'https://as.wikipedia.org/wiki/%E0%A6%A1%E0%A7%87%E0%A6%9F%E0%A6%BE_%E0%A6%AE%E0%A6%BE%E0%A6%87%E0%A6%A8%E0%A6%BF%E0%A6%82', 'https://az.wikipedia.org/wiki/Data_mining', 'https://bn.wikipedia.org/wiki/%E0%A6%89%E0%A6%AA%E0%A6%BE%E0%A6%A4%E0%A7%8D%E0%A6%A4_%E0%A6%96%E0%A6%A8%E0%A6%A8', 'https://bg.wikipedia.org/wiki/%D0%98%D0%B7%D0%B2%D0%BB%D0%B8%D1%87%D0%B0%D0%BD%D0%B5_